# Planner optimality study

This notebook compares QuASAr's planner decisions with an exhaustive search over backend/hybrid
choices for small circuits. It reports the cost gap between the heuristic plan (which may use the
planner's quick path) and the globally optimal plan derived from exhaustive enumeration.


## Overview

1. Generate a catalogue of small Clifford+rotation circuits.
2. Analyse each circuit into QuSD partitions.
3. Run QuASAr's planner with default settings.
4. Enumerate all planner options to obtain the optimal (minimum estimated cost) plan.
5. Summarise the optimality gap and export CSV/figures.


In [ ]:
from __future__ import annotations

import itertools
from pathlib import Path
import sys
from typing import Dict, Iterable, List

import numpy as np
try:
    import pandas as pd
except ImportError:  # pragma: no cover - optional dependency
    pd = None
import matplotlib.pyplot as plt

REPO_ROOT = Path.cwd()
if (REPO_ROOT / 'scripts').exists() and str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from qiskit import QuantumCircuit
from qiskit.circuit.random import random_circuit

from scripts.calibration.common import random_tail_circuit, random_clifford_circuit, collect_metrics
from quasar.analyzer import analyze
from quasar.cost_estimator import CostEstimator
from quasar.qusd import Plan
from quasar.planner import PlannerConfig, plan as quasar_plan
import quasar.planner as planner_module

plt.rcParams.update({'figure.figsize': (7.5, 4.5), 'axes.grid': True})


In [ ]:
def generate_test_circuits() -> List[QuantumCircuit]:
    circuits: List[QuantumCircuit] = []
    seeds = np.random.default_rng(13).integers(0, 2**31 - 1, size=16, dtype=np.int64)
    # Structured Clifford+rotation tails.
    for idx, seed in enumerate(seeds[:6]):
        n = 2 + idx % 3
        layers = 3 + idx % 2
        circ = random_tail_circuit(
            n=n,
            layers=layers,
            angle_scale=0.2,
            rotation_prob=0.5,
            twoq_prob=0.6,
            branch_prob=0.25,
            diag_prob=0.2,
            seed=int(seed),
        )
        circuits.append(circ)
    # Pure Clifford circuits.
    for idx, seed in enumerate(seeds[6:12]):
        n = 2 + idx % 2
        depth = 4 + idx
        circuits.append(random_clifford_circuit(n=n, depth=depth, seed=int(seed)))
    # Random Qiskit circuits to stress heuristic splits.
    for idx, seed in enumerate(seeds[12:]):
        circ = random_circuit(num_qubits=3 + (idx % 2), depth=4 + idx, measure=False, seed=int(seed))
        circuits.append(circ)
    # Add a handcrafted example with an obvious hybrid split.
    manual = QuantumCircuit(3)
    manual.h(0)
    manual.cx(0, 1)
    manual.cx(1, 2)
    manual.t(2)
    manual.cz(0, 2)
    manual.rx(0.2, 2)
    manual.cz(1, 2)
    circuits.append(manual)
    return circuits


def exhaustive_plan(base_plan: Plan, cfg: PlannerConfig) -> Plan:
    estimator = CostEstimator.from_planner_config(cfg)
    option_sets: List[List[List]] = []
    for node in base_plan.qusds:
        options = planner_module._plan_options_for_partition(node, cfg)
        if not options:
            options = [[]]
        option_sets.append(options)
    meta = dict(base_plan.meta)
    best_plan: Plan | None = None
    for combination in itertools.product(*option_sets) if option_sets else [()]:
        candidate = Plan(meta=dict(meta))
        for option in combination:
            if not option:
                continue
            candidate.extend_plan(option, estimator)
        if best_plan is None or candidate.estimated_cost < best_plan.estimated_cost:
            best_plan = candidate
    return best_plan if best_plan is not None else Plan(meta=dict(meta))


In [ ]:
cfg = PlannerConfig()
circuits = generate_test_circuits()
len(circuits)


In [ ]:
records: List[Dict[str, object]] = []
for idx, circuit in enumerate(circuits):
    analysis = analyze(circuit)
    base_plan = analysis.plan
    metrics = analysis.metrics_global
    quick_path = planner_module._should_use_quick_path(base_plan, cfg)
    heuristic = quasar_plan(base_plan, cfg)
    optimal = exhaustive_plan(base_plan, cfg)
    optimal_cost = optimal.estimated_cost or 0.0
    heuristic_cost = heuristic.estimated_cost or 0.0
    gap = float('nan')
    if optimal_cost > 0:
        gap = (heuristic_cost - optimal_cost) / optimal_cost
    record = {
        'circuit_id': idx,
        'num_qubits': circuit.num_qubits,
        'depth': circuit.depth(),
        'num_gates': metrics.get('num_gates'),
        'two_qubit_gates': metrics.get('two_qubit_gates'),
        'rotation_count': metrics.get('rotation_count'),
        'heuristic_cost': heuristic_cost,
        'optimal_cost': optimal_cost,
        'gap_fraction': gap,
        'heuristic_steps': len(heuristic.qusds),
        'optimal_steps': len(optimal.qusds),
        'quick_path': bool(quick_path),
        'heuristic_trace': heuristic.decision_trace,
        'optimal_trace': optimal.decision_trace,
    }
    records.append(record)
len(records)


In [ ]:
if pd is not None:
    gap_df = pd.DataFrame(records)
    display(gap_df.head())
else:
    gap_df = records
    gap_df[:3]


In [ ]:
if pd is None:
    raise RuntimeError('Install pandas to continue the analysis.')
summary = gap_df[['gap_fraction', 'quick_path']].dropna()
avg_gap = summary['gap_fraction'].mean() if not summary.empty else float('nan')
max_gap = summary['gap_fraction'].max() if not summary.empty else float('nan')
avg_gap, max_gap


In [ ]:
results_dir = Path('final_results')
results_dir.mkdir(parents=True, exist_ok=True)
gap_csv = results_dir / 'planner_optimality_gap.csv'
gap_df.to_csv(gap_csv, index=False)
gap_csv


In [ ]:
fig, ax = plt.subplots()
subset = gap_df.dropna(subset=['gap_fraction'])
ax.bar(subset['circuit_id'], subset['gap_fraction'])
ax.set_xlabel('Circuit index')
ax.set_ylabel('Relative gap')
ax.set_title('Heuristic vs. optimal estimated cost gap')
fig.tight_layout()
plot_path = Path('plots') / 'planner_optimality_gap.png'
plot_path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(plot_path, dpi=150)
plot_path
